# Multimodal Travel Planner

In [ ]:
import os
import requests
import pickle 
import math
import folium
import pandas as pd
from dotenv import load_dotenv
import googlemaps

# load API keys
load_dotenv()
maps_api_key = os.getenv("MAPS_API_KEY")
amadeus_api_key = os.getenv("AMADEUS_API_KEY")
amadeus_api_secret = os.getenv("AMADEUS_API_SECRET")

gmaps = googlemaps.Client(key=maps_api_key)


In [29]:
def get_amadeus_token():
    response = requests.post(
        'https://test.api.amadeus.com/v1/security/oauth2/token',
        data={
            'grant_type': 'client_credentials',
            'client_id': amadeus_api_key,
            'client_secret': amadeus_api_secret
        }
    )
    return response.json().get("access_token", None)


In [34]:
# Static airport coordinates
airports = {
    "DEN": ("Denver International", (39.8561, -104.6737)),
    "SLC": ("Salt Lake City Intl", (40.7899, -111.9791)),
    "SMF": ("Sacramento Intl", (38.6951, -121.5908)),
    "LAX": ("Los Angeles Intl", (33.9416, -118.4085)),
    "DTW": ("Detroit Metro", (42.2162, -83.3554)),
    "ZRH": ("Zurich", (47.4581, 8.5555)),
    "GVA": ("Geneva", (46.2381, 6.1096)),
    "INN": ("Innsbruck", (47.2602, 11.3430)),
    "CDG": ("Paris Charles de Gaulle", (49.0097, 2.5479)),
    "MSP": ("Minneapolis-St Paul", (44.8848, -93.2223))
}

# Nearest airport
def nearest_airport(coord):
    best_code, best_name, best_dist = None, None, float("inf")
    for code, (name, a_coord) in airports.items():
        dist = haversine(coord, a_coord)
        if dist < best_dist:
            best_code, best_name, best_dist = code, name, dist
    return best_code, (best_name, airports[best_code][1])

# Flat flight estimate
def estimate_flight_price(distance_miles):
    return round(0.35 * distance_miles + 50, 2)

In [35]:
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c * 0.621371

def estimate_cost(mode, distance):
    if mode == "driving": return distance * 0.5
    elif mode == "transit": return distance * 0.2 + 2
    elif mode == "flight": return 50 + distance * 0.15
    return 0

def nearest_airport(coord, airport_dict):
    return min(airport_dict.items(), key=lambda x: haversine(coord, x[1][1]))


In [36]:
with open("/Users/laurenbair/Documents/GitHub/FinalProject/paths_route.pkl", "rb") as f:
    route_dict = pickle.load(f)

print(route_dict)

{'Denver Airport': (39.8571391027151, -104.67682422525337), 'Eldora': (39.937376571357305, -105.58271078989766), 'A-Basin': (39.634289749362566, -105.8714993899124), 'Keystone': (39.58186424927065, -105.94362588806374), 'Breckenridge': (39.48111351412133, -106.07307234912773), 'Vail': (39.60634275473704, -106.35502534758554), 'Beaver Creek': (39.601840601122596, -106.53163266107806), 'Canyons': (40.685834569379516, -111.55630231684529), 'Kirkwood': (38.68503611762454, -120.06520941879404), 'Heavenly': (38.9288904430737, -119.90519040528994), 'Northstar': (39.26481569676928, -120.13316534575092), 'Arlberg': (47.129520437009155, 10.263859008628906), 'Verbier': (46.09708737973264, 7.227272162830475), 'The 3 Valleys': (45.34154189826385, 6.586552723873957), 'Mt. Brighton': (42.5410196899345, -83.81160753394613), 'Afton Alps': (44.85782048122588, -92.78779547430095)}


In [37]:
journey_log = []

locations = list(route_dict.keys())
location_coords = route_dict

for i in range(len(locations) - 1):
    origin = locations[i]
    dest = locations[i + 1]
    origin_coords = location_coords[origin]
    dest_coords = location_coords[dest]

    origin_str = f"{origin_coords[0]},{origin_coords[1]}"
    dest_str = f"{dest_coords[0]},{dest_coords[1]}"

    best_mode = None
    best_cost = float('inf')
    best_distance = 0

    # Try driving and transit first
    for mode in ["transit", "driving"]:
        try:
            directions = gmaps.directions(origin_str, dest_str, mode=mode)
            if directions:
                leg = directions[0]['legs'][0]
                distance = leg['distance']['value'] * 0.000621371
                cost = estimate_cost(mode, distance)
                if cost < best_cost:
                    best_cost = cost
                    best_distance = distance
                    best_mode = mode
        except Exception as e:
            print(f"Error fetching {mode} directions from {origin} to {dest}: {e}")
            continue

    # ✈️ Try flight if long distance
    direct_distance = haversine(origin_coords, dest_coords)
    if direct_distance > 300:
        try:
            o_code, (o_name, o_coords) = nearest_airport(origin_coords, airports)
            d_code, (d_name, d_coords) = nearest_airport(dest_coords, airports)

            try:
                flight_price = get_flight_price(o_code, d_code)
                if flight_price is None:
                    raise ValueError("No price from Amadeus")
            except:
                flight_price = estimate_flight_price(direct_distance)

            ground1 = gmaps.directions(origin_str, o_name, mode="driving")[0]['legs'][0]
            ground2 = gmaps.directions(d_name, dest_str, mode="driving")[0]['legs'][0]

            total_flight = (
                float(flight_price) +
                estimate_cost("driving", ground1['distance']['value'] * 0.000621371) +
                estimate_cost("driving", ground2['distance']['value'] * 0.000621371)
            )

            if total_flight < best_cost:
                journey_log.extend([
                    {"from": origin, "to": o_name, "mode": "driving",
                     "distance_miles": round(ground1['distance']['value'] * 0.000621371, 2),
                     "cost_usd": round(estimate_cost("driving", ground1['distance']['value'] * 0.000621371), 2)},
                    {"from": f"{o_code} Airport", "to": f"{d_code} Airport", "mode": "flight",
                     "distance_miles": round(direct_distance, 2),
                     "cost_usd": round(float(flight_price), 2)},
                    {"from": d_name, "to": dest, "mode": "driving",
                     "distance_miles": round(ground2['distance']['value'] * 0.000621371, 2),
                     "cost_usd": round(estimate_cost("driving", ground2['distance']['value'] * 0.000621371), 2)}
                ])
                continue
        except Exception as e:
            print(f"Flight fallback failed for {origin} to {dest}: {e}")

    # 🚗 Fallback if no good route
    if best_mode is None:
        print(f"No route found from {origin} to {dest}. Setting fallback values.")
        journey_log.append({
            "from": origin,
            "to": dest,
            "mode": "unavailable",
            "distance_miles": 0.0,
            "cost_usd": float('inf')
        })
    else:
        journey_log.append({
            "from": origin,
            "to": dest,
            "mode": best_mode,
            "distance_miles": round(best_distance, 2),
            "cost_usd": round(best_cost, 2)
        })


In [38]:
df = pd.DataFrame(journey_log)
df["leg"] = df.index + 1
print(df)
print(f"Total Cost: ${df['cost_usd'].sum():.2f}")
print(f"Total Distance: {df['distance_miles'].sum():.2f} mi")


                   from                   to     mode  distance_miles  \
0        Denver Airport               Eldora  transit           67.24   
1                Eldora              A-Basin  transit          141.99   
2               A-Basin             Keystone  driving            7.56   
3              Keystone         Breckenridge  driving           18.20   
4          Breckenridge                 Vail  driving           44.57   
5                  Vail         Beaver Creek  driving           22.51   
6          Beaver Creek              Canyons  transit          728.80   
7               Canyons  Salt Lake City Intl  driving           27.41   
8           SLC Airport          SMF Airport   flight          472.85   
9       Sacramento Intl             Kirkwood  driving           96.55   
10             Kirkwood             Heavenly  driving           40.05   
11             Heavenly            Northstar  driving           45.05   
12            Northstar      Sacramento Intl  drivi

In [53]:
from tabulate import tabulate

print(tabulate(
    df[["from", "to", "mode", "distance_miles", "cost_usd"]],
    headers="keys",
    tablefmt="grid",
    showindex=True
))

total_cost = df["cost_usd"].sum()
total_distance = df["distance_miles"].sum()

print("\n" + "="*60)
print(f"Total Cost: ${total_cost:,.2f}")
print(f"Total Distance: {total_distance:,.2f} miles")
print("="*60)

+----+---------------------+---------------------+---------+------------------+------------+
|    | from                | to                  | mode    |   distance_miles |   cost_usd |
+====+=====================+=====================+=========+==================+============+
|  0 | Denver Airport      | Eldora              | transit |            67.24 |      15.45 |
+----+---------------------+---------------------+---------+------------------+------------+
|  1 | Eldora              | A-Basin             | transit |           141.99 |      30.4  |
+----+---------------------+---------------------+---------+------------------+------------+
|  2 | A-Basin             | Keystone            | driving |             7.56 |       3.78 |
+----+---------------------+---------------------+---------+------------------+------------+
|  3 | Keystone            | Breckenridge        | driving |            18.2  |       9.1  |
+----+---------------------+---------------------+---------+----------

In [51]:
mode_icons = {
    "driving": "car",
    "transit": "train",
    "flight": "plane"
}

m = folium.Map(location=location_coords['Denver Airport'], zoom_start=5)
for leg in journey_log:
    start = leg["from"]
    end = leg["to"]
    start_coord = location_coords.get(start, None)
    end_coord = location_coords.get(end, None)
    for a in airports.values():
        if start == a[0]: start_coord = a[1]
        if end == a[0]: end_coord = a[1]
    if start_coord and end_coord:
        folium.Marker(start_coord, tooltip=start, icon=folium.Icon(color="blue", icon=mode_icons.get(leg["mode"], "info-sign"), prefix="fa")).add_to(m)
        folium.Marker(end_coord, tooltip=end, icon=folium.Icon(color="green", icon=mode_icons.get(leg["mode"], "info-sign"), prefix="fa")).add_to(m)
        folium.PolyLine([start_coord, end_coord], color="black" if leg["mode"] == "flight" else "blue").add_to(m)

m.save("multimodal_trip_map.html")
print("Map saved to multimodal_trip_map.html")


Map saved to multimodal_trip_map.html
